# 基于RAG技术的知识问答

数据来源：arXiv

任务：

    1. 输入用户问题
    2. 搜索向量存储库找出匹配的论文abstract

## 初始化

1. llm:Qwen2.5-14B模型

2. embedding
>`sentence-transformers/all-MiniLM-L12-v2` 是 `sentence-transformers` 项目下的一个很受欢迎的预训练模型。它擅长将句子级别的文本转换为高质量的向量表示，能够很好地捕捉句子的语义信息，并且生成的向量维度相对合理（通常在计算和存储成本上有较好的平衡）。
>
>这个模型在诸多文本相似性任务中表现出色，例如在文本聚类、语义搜索、问答匹配等场景下，可以基于它生成的向量快速准确地找到语义相近的文本内容

3. 数据arXiv下载（如果搞自己的本地离线向量数据库）

3. 向量数据库Milvus
>Milvus 是一款开源的、专门用于处理海量向量数据的数据库管理系统。它旨在高效地存储、索引以及快速检索高维向量数据，能够很好地支持各种基于向量相似度的应用，比如在自然语言处理领域的语义搜索、图像识别领域的图像特征匹配等场景中发挥重要作用。与传统的关系型数据库不同，它聚焦于向量这种数据形式，针对向量的特点优化了存储和查询的性能

In [25]:
# 初始化llm
from langchain.chat_models import ChatOpenAI
import os
llm_model = "Qwen2.5-14B"
os.environ["OPENAI_API_KEY"] = "None"
os.environ["OPENAI_API_BASE"] = "http://10.58.0.2:8000/v1"
# llm_completion = OpenAI(model_name="Qwen2.5-14B")
# llm_chat = OpenAIChat(model_name="Qwen2.5-14B")
llm = ChatOpenAI(temperature=0, model=llm_model)


In [2]:
# 初始化embedding方式
# !pip install sentence_transformers
# 每次都得翻墙很麻烦，直接离线到本地：
from langchain.embeddings import HuggingFaceEmbeddings
# embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2")
embedding = HuggingFaceEmbeddings(model_name="./all-MiniLM-L12-v2")

d:\Anaconda\envs\d2lc\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
# 初始化数据集下载 若需要搭建自己的向量数据库
# import kagglehub
# # Download latest version
# path = kagglehub.dataset_download("Cornell-University/arxiv")
# print("Path to dataset files:", path)

In [21]:
# 初始化向量数据库
# 不使用本地内存向量数据库，而是提供的在线向量数据库
# !pip install protobuf
# !pip install pymilvus
# 检索时，输入的数据也会先通过这个嵌入函数转换为向量，进而在数据库中查找相似向量

# from langchain.vectorstores import Milvus
# db = Milvus(embedding_function=embedding,collection_name='arXiv',connection_args={"host": "10.58.0.2", "port": "19530"})

from pymilvus import connections
connections.connect(
  host='10.58.0.2',
  port='19530'
)

from pymilvus import utility
print(utility.has_collection("arXiv"))
from langchain.vectorstores import Milvus
db = Milvus(embedding_function=embedding,collection_name='arxiv',connection_args={"host": "10.58.0.2", "port": "19530"})
# 深坑，这里的arxiv的x不能大写

True


## 简单测试

简单查询向量数据库相关文档

In [26]:
# 形成QA应用
from langchain.chains import RetrievalQA
retriever = db.as_retriever()
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [31]:
from pymilvus import Collection
collection = Collection("arxiv")      # Get an existing collection.
collection.load()

res = db.query(
  expr = "deep learning",
  offset = 0,
  limit = 10, 
  output_fields = ["text", "comments"],
)

AttributeError: 'Milvus' object has no attribute 'query'

In [30]:
qa_stuff.run("deep learning")



> Entering new RetrievalQA chain...

> Finished chain.


"The provided context does not discuss deep learning. It focuses on statistical learning, specifically the construction of predictors for a random variable $Y$ based on another random variable $X$, under conditions where the $Y$-part of the data is communicated at a finite bit rate. If you have questions about deep learning, I would be happy to say that I don't know and help find accurate information on that topic."